<h1><center>Laboratorio 6: La solicitud de Sergio 🤗</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

Las celdas de este archivo no están ejecutadas debido al limite de MB de GitHub



### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristian Oyarzo M.
- Nombre de alumno 2: Sebastián Quenti A.


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/sebaquenti/Repositorio-MDS7202)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers.
- Utilizar diferentes algoritmos de cluster y ver el desempeño.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar clusters.
- Familiarizarse con plotly.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio

<center>
<img src="https://i.pinimg.com/originals/5a/a6/af/5aa6afde8490da403a21601adf7a7240.gif" width=400 />

En el corazón de las operaciones de Aerolínea Lucero, Sergio, el gerente de análisis de datos, reunió a un talentoso equipo de jóvenes científicos de datos para un desafío crucial: segmentar la base de datos de los clientes. “Nuestro objetivo es descubrir patrones en el comportamiento de los pasajeros que nos permitan personalizar servicios y optimizar nuestras campañas de marketing,” explicó Sergio, mientras desplegaba un amplio rango de datos que incluían desde hábitos de compra hasta opiniones sobre los vuelos.

Sergio encargó a los científicos de datos la tarea de aplicar técnicas avanzadas de clustering para identificar distintos segmentos de clientes, como los viajeros frecuentes y aquellos que eligen la aerolínea para celebrar ocasiones especiales. La meta principal era entender profundamente cómo estos grupos perciben la calidad y satisfacción de los servicios ofrecidos por la aerolínea.

A través de un enfoque meticuloso y colaborativo, los científicos de datos se abocaron a la tarea, buscando transformar los datos brutos en valiosos insights que permitirían a Aerolínea Lucero no solo mejorar su servicio, sino también fortalecer las relaciones con sus clientes mediante una oferta más personalizada y efectiva.

## Importamos librerias utiles 😸

In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import time

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest

## 1. Estudio de Performance 📈 [10 Puntos]



<center>
<img src="https://user-images.githubusercontent.com/57133330/188281408-c67df9ee-fd1f-4b37-833b-f02848f1ce02.gif" width=300>

Don Sergio les ha encomendado su primera tarea: analizar diversas técnicas de clustering. Su objetivo es entender detalladamente cómo funcionan estos métodos en términos de segmentación y eficiencia en tiempo de ejecución.

Analice y compare el desempeño, tiempo de ejecución y visualizaciones de cuatro algoritmos de clustering (k-means, DBSCAN, Ward y GMM) aplicados a tres conjuntos de datos, incrementando progresivamente su tamaño. Utilice Plotly para las gráficas y discuta los resultados tanto cualitativa como cuantitativamente.

Uno de los requisitos establecidos por Sergio es que el análisis se lleve a cabo utilizando Plotly; de no ser así, se considerará incorrecto. Para facilitar este proceso, se ha proporcionado un código de Plotly que puede servir como base para realizar las gráficas. Apóyese en el código entregado para efectuar el análisis y tome como referencia la siguiente imagen para realizar los gráficos:

<img src='https://gitlab.com/imezadelajara/datos_clase_7_mds7202/-/raw/main/misc_images/Screenshot_2024-04-26_at_9.10.44_AM.png' width=800 />

En el gráfico se visualizan en dos dimensiones los diferentes tipos de datos proporcionados en `datasets`. Cada columna corresponde a un modelo de clustering diferente, mientras que cada fila representa un conjunto de datos distinto. Cada uno de los gráficos incluye el tiempo en segundos que tarda el análisis y la métrica Silhouette obtenida.

Para ser más específicos, usted debe cumplir los siguientes objetivos:
1. Generar una función que permita replicar el gráfico expuesto en la imagen (no importa que los colores calcen). [4 puntos]
2. Ejecuta la función para un `n_samples` igual a 1000, 5000, 10000. [2 puntos]
3. Analice y compare el desempeño, tiempo de ejecución y visualizaciones de cuatro algoritmos de clustering utilizando las 3 configuraciones dadas en `n_samples`. [4 puntos]


> ❗ Tiene libertad absoluta de escoger los hiper parámetros de los cluster, sin embargo, se recomienda verificar el dominio de las variables para realizar la segmentación.

> ❗ Recuerde que es obligatorio el uso de plotly.


In [2]:
"""
En la siguiente celda se crean los datos ficticios a usar en la sección 1 del lab.
❗No realice cambios a esta celda a excepción de n_samples❗
"""

# Datos a utilizar

def create_data(n_samples):

    # Lunas
    moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=30)
    # Blobs
    blobs = datasets.make_blobs(n_samples=n_samples, random_state=172)
    # Datos desiguales
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    mutated = (np.dot(blobs[0], transformation), blobs[1])
    
    # Generamos Dataset
    dataset = {
        'moons':{
            'x': moons[0], 'classes': moons[1], 'n_cluster': 2
        },
        'blobs':{
            'x': blobs[0], 'classes': blobs[1], 'n_cluster': 3
        },
        'mutated':{
            'x': mutated[0], 'classes': mutated[1], 'n_cluster': 3
        }
    }
    return dataset

data_sets_1000 = create_data(1000)
data_sets_5000 = create_data(5000)
data_sets_10000 = create_data(10000)

**Respuestas:**

In [3]:
# Función auxiliar para realizar los clusters
def apply_clustering(algorithm_name, X, n_clusters):    
    start_time = time.time()
    
    if algorithm_name == 'kmeans':
        model = KMeans(n_clusters=n_clusters, random_state=1323, n_init=10)

        labels = model.fit_predict(X)
    
    elif algorithm_name == 'dbscan':
        model = DBSCAN(eps=0.4, min_samples=3)
        labels = model.fit_predict(X)
    
    elif algorithm_name == 'ward':
        model = AgglomerativeClustering(n_clusters=n_clusters)
        labels = model.fit_predict(X)
    
    elif algorithm_name == 'gmm':
        model = GaussianMixture(n_components=n_clusters, random_state=1323)
        labels = model.fit_predict(X)
    
    else:
        raise ValueError(f"Algoritmo {algorithm_name} no reconocido.")
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    # Silhouette score solo se puede calcular si hay más de un clúster
    if len(set(labels)) > 1:
        silhouette = silhouette_score(X, labels)
    else:
        silhouette = -1  # Valor inválido si hay un solo clúster
    
    return labels, silhouette, execution_time

In [4]:
# Función para graficar la cuadrícula
def plot_scatter(datasets, n_samples):
    algorithms = ['kmeans', 'gmm', 'ward', 'dbscan']
    algo_titles = ["Kmeans", "GMM", "WARD", "DBSCAN"]
    fig = make_subplots(
        rows=3, cols=4, subplot_titles=[f"{algo}" for algo in algo_titles * 3], 
        horizontal_spacing=0.05, vertical_spacing=0.15
    )
    
    row = 1
    for dataset_name, data in datasets.items():
        X = data['x']
        n_clusters = data['n_cluster']
        
        for col, algo in enumerate(algorithms, 1):
            labels, silhouette, execution_time = apply_clustering(algo, X, n_clusters)
            scatter = go.Scatter(
                x=X[:, 0], y=X[:, 1], mode='markers',
                marker=dict(color=labels, colorscale='Viridis', size=5),
                name=f'{dataset_name} - {algo}',
                showlegend=False
            )
            
            # Añadir el scatter plot en la posición adecuada
            fig.add_trace(scatter, row=row, col=col)
            # Añadir el título con los tiempos y la métrica Silhouette
            fig.update_xaxes(title_text=f"{execution_time:.2f} [s] | s: {silhouette:.2f}", row=row, col=col)

        row += 1

    fig.update_layout(height=800, width=1000, title_text=f"Comparación de tiempos de ejecución y Silhouette por técnica (n={n_samples})")
    fig.show()

In [ ]:
# Graficar para 1000 muestras con la nueva función de cuadrícula
plot_scatter(data_sets_1000, 1000)

# Graficar para 5000 muestras
plot_scatter(data_sets_5000, 5000)

# Graficar para 10000 muestras
plot_scatter(data_sets_10000, 10000)

En general, a medida que aumentó el tamaño de las muestras, los tiempos de ejecución se incrementaron en todos los algoritmos, pero la calidad de los clústers, medida por el índice de Silhouette, se mantuvo relativamente constante. Los tiempos de ejecución más notables se observaron en WARD, mientras que Kmeans y GMM mostraron tiempos mucho más reducidos. En general, los algoritmos fueron consistentes en su desempeño respecto a la estructura de los datos a través de los diferentes tamaños de muestra.

Kmeans mostró un buen desempeño en términos de velocidad, siendo rápido en todas las configuraciones. Sin embargo, tuvo dificultades para adaptarse a clústers con formas no circulares, como el conjunto de datos "moons". Su índice de Silhouette fue alto en clústers bien separados, pero cayó cuando se enfrentó a formas más complejas.

GMM tuvo un desempeño similar a Kmeans en cuanto a la calidad de los clústers y mantuvo un tiempo de ejecución aún más bajo. Al igual que Kmeans, funcionó bien con clústers esféricos, pero no fue capaz de capturar adecuadamente las estructuras más complejas. A pesar de ello, sigue siendo uno de los modelos más rápidos, incluso con tamaños de muestra grandes.

WARD fue el algoritmo con mayor costo en tiempo de ejecución, especialmente con tamaños de muestra mayores. Si bien ofrece buenos resultados en clústers simples, su capacidad para detectar clústers no esféricos es limitada, como se vio en los datos "moons". Debido a su mayor tiempo de ejecución, es menos eficiente que otros algoritmos cuando se trabaja con grandes cantidades de datos.

DBSCAN en términos del índice de Silhouette, consistentemente tuvo un desempeño inferior en comparación con los otros algoritmos. Esto se debe a su manejo del ruido, que tiende a afectar negativamente esta métrica.

## 2. Análisis de Satisfacción de Vuelos. [10 puntos]

<center>
<img src="https://i.gifer.com/2Hci.gif" width=400 />

Habiendo entendido cómo funcionan los modelos de aprendizaje no supervisado, *Don Sergio* le encomienda estudiar la satisfacción de pasajeros al haber tomado un vuelo en alguna de sus aerolineas. Para esto, el magnate le dispone del dataset `aerolineas_licer.parquet`, el cual contiene el grado de satisfacción de los clientes frente a diferentes aspectos del vuelo. Las características del vuelo se definen a continuación:

- *Gender*: Género de los pasajeros (Femenino, Masculino)
- *Customer Type*: Tipo de cliente (Cliente habitual, cliente no habitual)
- *Age*: Edad actual de los pasajeros
- *Type of Travel*: Propósito del vuelo de los pasajeros (Viaje personal, Viaje de negocios)
- *Class*: Clase de viaje en el avión de los pasajeros (Business, Eco, Eco Plus)
- *Flight distance*: Distancia del vuelo de este viaje
- *Inflight wifi service*: Nivel de satisfacción del servicio de wifi durante el vuelo (0:No Aplicable; 1-5)
- *Departure/Arrival time convenient*: Nivel de satisfacción con la conveniencia del horario de salida/llegada
- *Ease of Online booking*: Nivel de satisfacción con la facilidad de reserva en línea
- *Gate location*: Nivel de satisfacción con la ubicación de la puerta
- *Food and drink*: Nivel de satisfacción con la comida y la bebida
- *Online boarding*: Nivel de satisfacción con el embarque en línea
- *Seat comfort*: Nivel de satisfacción con la comodidad del asiento
- *Inflight entertainment*: Nivel de satisfacción con el entretenimiento durante el vuelo
- *On-board service*: Nivel de satisfacción con el servicio a bordo
- *Leg room service*: Nivel de satisfacción con el espacio para las piernas
- *Baggage handling*: Nivel de satisfacción con el manejo del equipaje
- *Check-in service*: Nivel de satisfacción con el servicio de check-in
- *Inflight service*: Nivel de satisfacción con el servicio durante el vuelo
- *Cleanliness*: Nivel de satisfacción con la limpieza
- *Departure Delay in Minutes*: Minutos de retraso en la salida
- *Arrival Delay in Minutes*: Minutos de retraso en la llegada

En consideración de lo anterior, realice las siguientes tareas:

0. Ingeste el dataset a su ambiente de trabajo.

1. Seleccione **sólo las variables numéricas del dataset**.  Explique qué éfectos podría causar el uso de variables categóricas en un algoritmo no supervisado. [2 punto]

2. Realice una visualización de la distribución de cada variable y analice cada una de estas distribuciones. [2 punto]

3. Basándose en los gráficos, evalúe la necesidad de escalar los datos y explique el motivo de su decisión. [2 puntos]

4. Examine la correlación entre las variables mediante un correlograma. [2 puntos]

5. De acuerdo con los resultados obtenidos en 5, reduzca la dimensionalidad del conjunto de datos a cuatro variables, justificando su elección respecto a las variables que decide eliminar. [2 puntos]

**Respuesta:**

In [ ]:
# Carga de datos
df = pd.read_parquet('aerolineas_lucer.parquet')

# Dimensiones del dataset
print(df.shape)

# Muestra las primeras filas del DataFrame
df.head()

In [ ]:
# 1
df_numericas = df.select_dtypes(include=['number'])

df_numericas.head()

In [ ]:
df_numericas.shape

1. Las variables categóricas pueden afectar negativamente el rendimiento de un algoritmo no supervisado porque muchos de estos algoritmos, como Kmeans o GMM, dependen de medidas de distancia, que no se aplican directamente a categorías. Si no se transforman correctamente (por ejemplo, con one-hot encoding), las categorías podrían generar distancias arbitrarias o malinterpretar la relación entre los datos, lo que resultaría en clústers poco representativos o confusos. Además, existen variables categóricas no jerárquicas, como el género, que pueden interferir en el análisis al no tener un orden inherente.

In [ ]:
# 2 
for column in df_numericas.columns:
    fig = px.histogram(df_numericas, x=column, title=f'Distribución de {column}', nbins=30)
    fig.show()

2. 
La variable ID presenta una distribución uniforme, como es esperable para este tipo de identificador.

La variable Age parece seguir una distribución aproximadamente normal, con una media cercana a los 40 años.

La distribución de Flight Distance abarca valores entre 0 y alrededor de 5000, con un pico notable entre 200 y 400. En contraste, las distancias superiores a 4000 tienen una frecuencia muy baja.

La variable Inflight wifi service toma valores entre 0 y 5, con 2 y 3 siendo los más frecuentes.

Departure/Arrival time convenient también varía entre 0 y 5, pero se distingue porque el valor más frecuente es 4, seguido de 5.

De manera similar, las variables Ease of online booking, Gate location, Food and drink, Online boarding, Seat comfort, Inflight entertainment, On-board service, Leg room service, Baggage handling, Check-in service, Inflight service y Cleanliness son discretas y toman valores entre 0 y 5.

Finalmente, Departure Delay y Arrival Delay presentan un comportamiento similar, con la mayoría de los valores comprendidos entre 0 y 100 minutos, y una marcada disminución de la frecuencia a partir de ese rango. En ambos casos, existen valores atípicos que superan los 1000 minutos.

3. Podemos observar que las variables en el dataset se miden en diferentes escalas, como minutos en el caso de los retrasos, o puntuaciones entre 0 y 5 para las valoraciones de servicio. Esta diversidad en las escalas puede afectar negativamente a los algoritmos basados en medidas de distancia, como K-means o GMM. Debido a esto será necesario escalar los datos más adelante.

In [ ]:
# 4

# Generar la matriz de correlación
correlation_matrix = df_numericas.corr()

# Crear el mapa de calor de la correlación con Plotly Express
fig = px.imshow(correlation_matrix, 
                labels=dict(x="Variables", y="Variables", color="Correlación"),
                title="Correlograma de variables numéricas",
                color_continuous_scale='Viridis')

# Ajustar el tamaño de la figura y la orientación de las etiquetas del eje X
fig.update_layout(
    width=1000,   
    height=1000, 
    title_font_size=24, 
)

# Hacer que las etiquetas del eje X queden verticales
fig.update_xaxes(tickangle=-90)

fig.show()

4. Se puede observar una importante correlación entre las variables, como las de delay o las relacionadas a la satisfacción de los clientes.

In [ ]:
df_numericas = df_numericas[['Age', 'Flight Distance', 'Inflight entertainment', 'Arrival Delay in Minutes']]

df_numericas.head()

5. Las variables seleccionadas fueron: primero, Age para caracterizar a los pasajeros; segundo, Flight Distance para incluir en el análisis el tipo de vuelo; tercero, Inflight entertainment como indicador de la satisfacción a bordo, dado que muestra una alta correlación con la mayoría de las variables discretas que toman valores entre 0 y 5. Finalmente, seleccionamos Arrival Delay in Minutes para evaluar los retrasos, ya que esta variable presenta una correlación cercana a 1 con el retraso en la partida (Departure Delay).

## 3. Preprocesamiento 🎭. [10 puntos]

<center>
<img src="https://i.pinimg.com/originals/1e/a8/0e/1ea80e7cea0d429146580c7e91c5b944.gif" width=400>

Tras quedar satisfecho con los resultados presentados en el punto 2, el dueño de la empresa ha solicitado que se preprocesen los datos mediante un `pipeline`. Es crucial que este proceso tenga en cuenta las observaciones derivadas de los análisis anteriores. Adicionalmente, ha expresado su interés en visualizar el conjunto de datos en un gráfico de dos o tres dimensiones.

Basándose en los análisis realizados anteriormente:
1. Cree un `pipeline` que incluya PCA, utilizando las consideraciones mencionadas previamente para proyectar los datos a dos dimensiones. [4 puntos]
2. Grafique los resultados obtenidos y comente lo visualizado. [6 puntos]

**Respuestas:**

In [ ]:
# Definir el pipeline con estandarización y PCA
pipeline_PCA = Pipeline([
    ('scaler', StandardScaler()),  # Escalar los datos
    ('pca', PCA(n_components=2))  # PCA para reducir a 2 dimensiones
])

# Aplicar el pipeline a los datos numéricos
data_reduced = pipeline_PCA.fit_transform(df_numericas)

# Crear un DataFrame con las dos dimensiones resultantes del PCA
df_pca = pd.DataFrame(data_reduced, columns=['PC1', 'PC2'])

# Graficar los resultados obtenidos en dos dimensiones
fig = px.scatter(df_pca, x='PC1', y='PC2', title='Proyección PCA a 2 dimensiones')

fig.show()

La mayor parte de los datos se concentra en valores bajos de PC2 (por debajo de 10), mientras que PC1 tiene un rango más amplio, lo que sugiere que esta componente captura la mayor parte de la variabilidad.

La dispersión de los puntos aumenta a medida que PC1 crece, mientras que algunos outliers se alejan hacia valores altos de PC2, indicando patrones inusuales en los datos.

## 4. Outliers 🚫🙅‍♀️❌🙅‍♂️ [10 puntos]

<center>
<img src="https://joachim-gassen.github.io/images/ani_sim_bad_leverage.gif" width=250>

Con el objetivo de mantener la claridad en su análisis, Don Sergio le ha solicitado entrenar un modelo que identifique pasajeros con comportamientos altamente atípicos.

1. Utilice `IsolationForest` para clasificar las anomalías del dataset (sin aplicar PCA), configurando el modelo para que sólo el 1% de los datos sean considerados anómalos. Asegúrese de integrar esta tarea dentro de un `pipeline`. [3 puntos]

2. Visualice los resultados en el gráfico de dos dimensiones previamente creado. [3 puntos]

3. ¿Cómo evaluaría el rendimiento de su modelo en la detección de anomalías? [4 puntos]

**Respuestas:**

In [ ]:
# Definir el pipeline con estandarización y Isolation Forest
pipeline_anomaly = Pipeline([
    ('isolation_forest', IsolationForest(contamination=0.01, random_state=1323))  # Modelar con 1% de anomalías
])

# Entrenar el modelo
pipeline_anomaly.fit(df_numericas)

# Predecir si cada punto es anómalo (-1) o no (1)
df_numericas['anomaly'] = pipeline_anomaly['isolation_forest'].predict(df_numericas)

# Reiniciar los índices de df_numericas y df_pca para que coincidan
df_numericas = df_numericas.reset_index(drop=True)
df_pca = df_pca.reset_index(drop=True)

# Añadir la columna de anomalía al DataFrame de PCA
df_pca['anomaly'] = df_numericas['anomaly']
df_pca['anomaly'] = df_pca['anomaly'].astype(str)

# Visualización de los resultados en 2D con colores que representen las anomalías
fig = px.scatter(df_pca, x='PC1', y='PC2', color='anomaly', 
                 title='Detección de anomalías usando Isolation Forest (Anomalías en rojo)')

fig.show()

Gráficamente se puede observar que los valores anómalos en el gráfico PCA son principalmente valores superiores a 6 en el eje PC2. Además los valores atípicos en el gráfico son detectados correctamente como anomalías. Sin embargo, algunos valores que en primera instancia con PCA no parecían anómalos si fueron identificados como tal. En general parece una detección de outliers aceptable.

## 5. Métricas de Desempeño 🚀 [10 puntos]

<center>
<img src="https://giffiles.alphacoders.com/219/219081.gif" width=300>

Motivado por incrementar su fortuna, Don Sergio le solicita entrenar un modelo que le permita segmentar a los pasajeros en grupos distintos, con el objetivo de optimizar las diversas campañas de marketing diseñadas por su equipo. Para ello, le se pide realizar las siguientes tareas:

1. Utilizar el modelo **Gaussian Mixture** y explore diferentes configuraciones de número de clústers, específicamente entre 3 y 8. Asegúrese de integrar esta operación dentro de un `pipeline`. [4 puntos]
2. Explique cuál sería el criterio adecuado para seleccionar el número óptimo de clústers. **Justifique de forma estadistica y a traves de gráficos.** [6 puntos]

> **HINT:** Se recomienda investigar sobre los criterios AIC y BIC para esta tarea.

**Respuestas:**

In [ ]:
# Convertir a array NumPy para evitar warnings
df_numericas_values = df_numericas.values

# Definir el rango de clústers a explorar
n_components_range = range(3, 9)
aic_values = []
bic_values = []

# Explorar diferentes configuraciones de Gaussian Mixture entre 3 y 8 clústers
for n_components in n_components_range:
    pipeline_gmm = Pipeline([
        ('scaler', StandardScaler()),  # Estandarización de los datos
        ('gmm', GaussianMixture(n_components=n_components, random_state=1323))
    ])
    
    # Ajustar el modelo utilizando el array NumPy
    pipeline_gmm.fit(df_numericas_values)
    
    # Obtener el modelo GMM entrenado
    gmm_model = pipeline_gmm.named_steps['gmm']
    
    # Guardar los valores de AIC y BIC
    aic_values.append(gmm_model.aic(df_numericas_values))
    bic_values.append(gmm_model.bic(df_numericas_values))

# Crear un DataFrame para los resultados
results = pd.DataFrame({
    'Número de Clústers': list(n_components_range),
    'AIC': aic_values,
    'BIC': bic_values
})

# Graficar AIC y BIC usando Plotly Express
fig = px.line(results, x='Número de Clústers', y=['AIC', 'BIC'], 
              title='Criterios AIC y BIC para seleccionar el número óptimo de clústers',
              labels={'value': 'Valor', 'variable': 'Criterio'},
              markers=True)
fig.show()

Al graficar los valores de AIC y BIC para cada número de clústers, se debe buscar el número donde ambos criterios alcanzan su valor mínimo. Este valor indica el mejor equilibrio entre ajuste del modelo y complejidad, evitando un modelo sobreajustado o demasiado simple.
Por lo tanto, el mejor número de clusters es 5

## 6. Análisis de resultados 📊 [10 puntos]

<center>
<img src="https://i.gifer.com/7wTk.gif" width=300>

Una vez identificado el número óptimo de clústers, se le pide realizar lo siguiente:

1. Utilizar la proyección en dos dimensiones para visualizar cada clúster claramente. [2 puntos]

2. ¿Es posible distinguir claramente entre los clústers generados? [2 puntos]

3. Proporcionar una descripción breve de cada clúster utilizando estadísticas descriptivas básicas, como la media y la desviación estándar, para resumir las características de las variables utilizadas en estos algoritmos. [2 puntos]

4. Proceda a visualizar los clústers en tres dimensiones para una perspectiva más detallada. [2 puntos]

5. ¿Cómo afecta esto a sus conclusiones anteriores? [2 puntos]

**Respuestas:**

In [ ]:
num_clusters = 5
df_numericas = df[['Age', 'Flight Distance', 'Inflight entertainment', 'Arrival Delay in Minutes']].copy() 

# Aplicar Gaussian Mixture con el número óptimo de clústers
gmm = GaussianMixture(n_components=num_clusters, random_state=1323)
gmm.fit(df_numericas)
df_numericas['cluster'] = gmm.predict(df_numericas)

# Proyección de los datos en 2 dimensiones usando PCA
data_pca = pipeline_PCA.fit_transform(df_numericas.drop(columns=['cluster']))
df_pca = pd.DataFrame(data_pca, columns=['PC1', 'PC2'])

# Reiniciar los índices de df_numericas y df_pca para que coincidan
df_numericas = df_numericas.reset_index(drop=True)
df_pca = df_pca.reset_index(drop=True)
df_pca['cluster'] = df_numericas['cluster']

# Visualizar los clústers en 2 dimensiones
df_pca['cluster'] = df_pca['cluster'].astype(str)
fig = px.scatter(df_pca, x='PC1', y='PC2', color='cluster', 
                 title=f'Visualización de clústers en 2D con {num_clusters} clústers')
fig.show()

2. Si bien el cluster 0 se puede ver fácilmente, el resto no se pueden distinguir entre ellos de manera clara, en particular todos estos clusters están alrededor del cero en PC2.

In [ ]:
# Generar estadísticas descriptivas de cada clúster
cluster_stats = df_numericas.groupby('cluster').agg(['mean', 'std'])
cluster_stats

In [ ]:
# Proyección de los datos en 3 dimensiones usando PCA
pca_3d = PCA(n_components=3)
data_pca_3d = pca_3d.fit_transform(df_numericas.drop(columns=['cluster']))
df_pca_3d = pd.DataFrame(data_pca_3d, columns=['PC1', 'PC2', 'PC3'])
df_pca_3d['cluster'] = df_numericas['cluster']

# Visualizar los clústers en 3 dimensiones
fig_3d = px.scatter_3d(df_pca_3d, x='PC1', y='PC2', z='PC3', color='cluster',
                       title=f'Visualización de clústers en 3D con {num_clusters} clústers')
fig_3d.show()

Nuevamente el cluster 0 es el más reconocible y el resto de cluster no se pueden observar de manera clara.

Como equipo creemos que quitar outliers podría generar mejores clusters y visualizaciones.